# Part 2: Data Modeling with Seasonal ARIMA (SARIMA)

In this part we will model one of the most popular Time-Series Forecasting model SARIMA, which is the subset of ARIMA.<br>

#   &emsp; &emsp;    A R I M A &emsp; ==> &emsp;    AR   &emsp;     &emsp;     +     &emsp;   &emsp;  MA

###                              &emsp;      &emsp;&emsp;&emsp;&emsp;&emsp;  &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;  (Auto Regression)    &emsp;     &emsp;    (Moving Average)
<H3>    &emsp; &emsp;&emsp;       I ==>    Integration </H3>





### Autoregression (AR)
The autoregression (AR) method models the next step in the sequence as a linear function of the observations at prior time steps.<br>

### Moving Average (MA)
A moving average (rolling average or running average) is a calculation to analyze data points by creating a series of averages of different subsets of the full data set. It is also called a moving mean (MM) or rolling mean and is a type of finite impulse response filter.

### Integration (I)
To make the time series stationary (by removing the Trends) by measuring the differences of observations at different time.


### Why not  ARIMA?
Although, Trends from the data can be removed by ARIMA model. But the series can still have seasonal component. Since, our data may have slight seasonal component So we will use SARIMA instead of ARIMA.
<br>

### SARIMA
<br>
Seasonal Autoregressive Integrated Moving Average, SARIMA or Seasonal ARIMA, is an extension of ARIMA that explicitly supports univariate time series data with a seasonal component.
<br>

***notation: <b>SARIMA(p,d,q)(P,D,Q,m)</b>*** <br><br>
Here (p,d,q) are Trend parameters and (P,D,Q,M) are seasonal parameters. Let's explore these parameters.<br>
<br><b>Trend parameter<br></b>
p: Trend autoregression order.<br>
d: Trend difference order.<br>
q: Trend moving average order.<br>
<br><b>Seasonal Parameter:<br></b>
P: Seasonal autoregressive order.<br>
D: Seasonal difference order.<br>
Q: Seasonal moving average order.<br>
m: The number of time steps for a single seasonal period.


In [1]:
# load datasets
